In [61]:
import requests
import json
import pandas as pd
import numpy as np

headers = {
	"x-rapidapi-key": "4bd63f5b60msh04536a28ff51c7ap1c420cjsn236a5e39a31a",
	"x-rapidapi-host": "nfl-api-data.p.rapidapi.com"
}

get_teams_url = "https://nfl-api-data.p.rapidapi.com/nfl-team-listing/v1/data"
get_players_from_teams_url = "https://nfl-api-data.p.rapidapi.com/nfl-team-roster"
get_player_stats_url = "https://nfl-api-data.p.rapidapi.com/nfl-ath-stats"

teams = requests.get(get_teams_url, headers=headers)
teams_json = teams.json()

def process_player_stats(data,player_name):
    # Extract categories and their statistics
    if 'categories' in data:
        

        categories = data['categories']
        # Prepare data for DataFrame
        stats_by_year = {}
        
        for category in categories:
            category_name = category['name']
            stat_names = category['names']
            
            for stat in category['statistics']:
                year = int(stat['season']['year'])  # Ensure year is an integer
                team = stat['teamSlug']  # Extract just the team name
                
                if year not in stats_by_year:
                    stats_by_year[year] = {'Team': team}
                
                for i, name in enumerate(stat_names):
                    column_name = f"{category_name}_{name}"
                    stats_by_year[year][column_name] = stat['stats'][i]
    
        # Create DataFrame
        df = pd.DataFrame.from_dict(stats_by_year, orient='index')
        df.index.name = 'Year'  # Set the index name to 'Year'
        df.reset_index(inplace=True)  # Make 'Year' a regular column
        df = df.sort_values('Year')  # Sort by year
    
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        df[numeric_columns] = df[numeric_columns].fillna(0)
        non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns
        df[non_numeric_columns] = df[non_numeric_columns].fillna("N/A")
        
        # Get the current team (last team in the most recent year)
        current_team = df.iloc[-1]['Team'] if not df.empty else "N/A"
        
        # Convert DataFrame to dict, replacing NaN with null
        stats_dict = df.where(pd.notnull(df), None).to_dict(orient='records')
        
        # Create the final JSON structure
        json_data = {
            "statistics_by_year": stats_dict
        }
    
        json_filename = f"{player_name}-{current_team}-stats.json"
        with open(json_filename, 'w') as json_file:
            json.dump(json_data, json_file, indent=2)
        print(f"Created file: {json_filename}")

    else:
        print("no data")

def extract_player_info(player):
    return {
        'player_id': player['id'],
        'Fullname': player['fullName'],
        'position': player['position']
    }

def is_valid_response(response):
    if response.status_code != 200:
        return False
    try:
        data = response.json()
        if isinstance(data, dict) and data.get("error") == "Request Failed Please try Again":
            return False
    except json.JSONDecodeError:
        return False
    return True

for team in teams_json:
        team_id = team['team']['id']
        querystring = {"id": team_id}
        team_players_by_team_id = requests.get(get_players_from_teams_url, headers=headers, params=querystring).json()
        players = team_players_by_team_id.get('athletes', [])

        players_info = [extract_player_info(player) for player in players]
        for player in players_info:
            player_id = player['player_id']
            fullname = player['Fullname']
            querystring = {"id": player_id}
            player_stats = requests.get(get_player_stats_url, headers=headers, params=querystring)
            if is_valid_response(player_stats):
                print(fullname)
                stats = player_stats.json()
                process_player_stats(stats,fullname)
            else:
                print(f"Failed to retrieve stats for player {fullname}. Status code: {player_stats.status_code}")
          
            
            

Failed to retrieve stats for player Isaiah Adams. Status code: 200
Jackson Barton
Created file: Jackson Barton-arizona-cardinals-stats.json
Kelvin Beachum
Created file: Kelvin Beachum-new-york-jets-stats.json
Trey Benson
Created file: Trey Benson-arizona-cardinals-stats.json
Failed to retrieve stats for player Evan Brown. Status code: 200
Trystan Colon
Created file: Trystan Colon-baltimore-ravens-stats.json
James Conner
Created file: James Conner-arizona-cardinals-stats.json
DeeJay Dallas
Created file: DeeJay Dallas-arizona-cardinals-stats.json
Emari Demercado
Created file: Emari Demercado-arizona-cardinals-stats.json
Greg Dortch
Created file: Greg Dortch-arizona-cardinals-stats.json
Hjalte Froholdt
Created file: Hjalte Froholdt-arizona-cardinals-stats.json
Failed to retrieve stats for player Jon Gaines II. Status code: 200
Marvin Harrison Jr.
Created file: Marvin Harrison Jr.-arizona-cardinals-stats.json
Charlie Heck
Created file: Charlie Heck-houston-texans-stats.json
Will Hernandez


In [ ]:
import json
import pandas as pd
import numpy as np

def process_player_stats(data):
    # Extract categories and their statistics
    categories = data['categories']
    
    # Get player name (you'll need to provide this as it's not in the given data)
    player_name = "Player"
    
    # Prepare data for DataFrame
    stats_by_year = {}
    
    for category in categories:
        category_name = category['name']
        stat_names = category['names']
        
        for stat in category['statistics']:
            year = int(stat['season']['year'])  # Ensure year is an integer
            team = stat['teamSlug']  # Extract just the team name
            
            if year not in stats_by_year:
                stats_by_year[year] = {'Team': team}
            
            for i, name in enumerate(stat_names):
                column_name = f"{category_name}_{name}"
                stats_by_year[year][column_name] = stat['stats'][i]

    # Create DataFrame
    df = pd.DataFrame.from_dict(stats_by_year, orient='index')
    df.index.name = 'Year'  # Set the index name to 'Year'
    df.reset_index(inplace=True)  # Make 'Year' a regular column
    df = df.sort_values('Year')  # Sort by year

    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].fillna(0)
    
    # Get the current team (last team in the most recent year)
    current_team = df.iloc[-1]['Team']

    stats_dict = df.to_dict(orient='records')
    
    # Create the final JSON structure
    json_data = {
        "statistics_by_year": stats_dict
    }

    json_filename = f"{player_name}-{current_team}-stats.json"
    with open(json_filename, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)
    print(f"Created file: {json_filename}")
    
    return json_data

player_data = player_data_json

# Process the player stats and get the DataFrame
player_stats_df = process_player_stats(player_data)

# Display the first few rows of the DataFrame

In [ ]:
player_stats_df